In [1]:
] activate juMLia

  Activating project at `d:\Users\gabri\Documents\Data Science & Machine Learning Spring 2025\Lundquist-CMOR438-Spring2025\Single Neuron (draft)\juMLia`


In [ ]:
import MLDatasets: Iris
using Plots, DataFrames
irisdata = Iris()
perceptrondata = subset(irisdata.dataframe, :class => species -> isequal.(species, "Iris-setosa") .|| isequal.(species, "Iris-versicolor"))
perceptrontargets = map(perceptrondata[:, :class]) do species
    if species == "Iris-setosa"
        return 1
    elseif species == "Iris-versicolor"
        return -1
    else
        return NaN
    end
end
perceptroninputs = perceptrondata[:, [:sepallength, :sepalwidth]]
setosadata = subset(irisdata.dataframe, :class => species -> isequal.(species, "Iris-setosa"))
versicolordata = subset(irisdata.dataframe, :class => species -> isequal.(species, "Iris-versicolor"))
include("Single Neuron (draft)/SingleNeuronJulia.jl")
plotdomain = range(minimum(perceptrondata[:, :sepallength]), maximum(perceptrondata[:, :sepallength]), length=100)
perceptronlinefunc(x, neur::SingleNeuron) = (neur.weights[1].*x .- neur.bias) ./ -neur.weights[2]
function plotperceptron(neur::SingleNeuron)
    scatter(setosadata[:, :sepallength], setosadata[:, :sepalwidth], label="Setosa", mc=:blue, ms=3)
    scatter!(versicolordata[:, :sepallength], versicolordata[:, :sepalwidth], label="Versicolor", mc=:magenta, ms=3)
    plot!(plotdomain, perceptronlinefunc(plotdomain, neur))
end


In [ ]:
perceptronmodel = SingleNeuron(2, :perceptron)
predict(perceptronmodel, perceptroninputs)
plotperceptron(perceptronmodel)

In [ ]:
train!(perceptronmodel, perceptroninputs, perceptrontargets; learningrate=0.05, numepochs=5)

In [ ]:
using MLDatasets
using Plots
using DataFrames

In [ ]:
irisdata = Iris()

In [ ]:
perceptrondata = subset(irisdata.dataframe, :class => species -> isequal.(species, "Iris-setosa") .|| isequal.(species, "Iris-versicolor"))
perceptrontargets = map(perceptrondata[:, :class]) do species
    if species == "Iris-setosa"
        return 1
    elseif species == "Iris-versicolor"
        return -1
    else
        return NaN
    end
end
perceptroninputs = perceptrondata[:, [:sepallength, :sepalwidth]]
setosadata = subset(irisdata.dataframe, :class => species -> isequal.(species, "Iris-setosa"))
versicolordata = subset(irisdata.dataframe, :class => species -> isequal.(species, "Iris-versicolor"))

In [ ]:
include("Single Neuron (draft)/SingleNeuronJulia.jl")

In [ ]:
plotdomain = range(minimum(perceptrondata[:, :sepallength]), maximum(perceptrondata[:, :sepallength]), length=100)
perceptronlinefunc(x, neur::SingleNeuron) = (neur.weights[1].*x .- neur.bias) ./ -neur.weights[2]
function plotperceptron(neur::SingleNeuron)
    scatter(setosadata[:, :sepallength], setosadata[:, :sepalwidth], label="Setosa", mc=:blue, ms=3)
    scatter!(versicolordata[:, :sepallength], versicolordata[:, :sepalwidth], label="Versicolor", mc=:magenta, ms=3)
    plot!(plotdomain, perceptronlinefunc(plotdomain, neur))
end

In [3]:
gr()

Plots.GRBackend()

In [4]:
include("SingleNeuronJulia.jl")

In [ ]:
perceptron_model = SingleNeuron(2, :perceptron)

SingleNeuron{typeof(sign_zeropositive), typeof(perceptronstochasticgradient), typeof(perceptronloss)}(Main.SingleNeuronJulia.sign_zeropositive, Main.SingleNeuronJulia.perceptronstochasticgradient, Main.SingleNeuronJulia.perceptronloss, [0.0, 0.0], 0.0, [0.0, 0.0], 0.0, Float64[], Float64[])

In [4]:
plotly()

Plots.PlotlyBackend()

In [3]:
x = range(0,10,100)
y = sin.(x)
p = plot(x,y)

In [ ]:
using LinearAlgebra

In [ ]:
"Returns -1 if the argument is less than 0, 1 otherwise."
sign_zeropositive(value) = value < 0.0 ? -1 : 1

"Returns the argument."
linear(value) = value

"""
Returns the value of the sigmoid function at input_value.
    
    https://en.wikipedia.org/wiki/Sigmoid_function 
"""
sigmoid(value) = 1.0 / (1.0 + exp(-value))


In [ ]:
mat = [1 2; 3 4]

In [ ]:
2size([1,2,3])[end]

In [ ]:
function perceptronloss(predictions, targets)
    return 0.25 * sum((predictions .- targets).^2)
end

function linearregressionloss(predictions::AbstractArray, targets::AbstractArray)
    return ( (1 / 2(size(targets)[end])) * sum((predictions .- targets).^2) )
end

function linearregressionloss(prediction::Number, target::Number)
    return ( 0.5 * (prediction - target)^2)
end

function binarycrossentropyloss(predictions::AbstractArray, targets::AbstractArray)
    return ( (1 / size(targets)[end]) 
            * sum(-targets .* log.(predictions)
                  - ((1 .- targets).*log.(1 .- predictions))) )
end

function binarycrossentropyloss(prediction::Number, target::Number)
    return ( -target * log(prediction)
                  - ((1 - target)*log(1 - prediction)) )
end

function meansquarederror(predictions, targets)
    return 0.5 .* (predictions .- targets).^2
end

In [ ]:
function perceptronstochasticgradient(prediction, target)
    return 0.5 * (prediction .- target)
end

function regressionstochasticgradient(prediction, target)
    return prediction .- target
end

In [ ]:
function preactivation(input, weights, bias)
    return dot(input, weights) + bias
end

In [ ]:
const type_perceptron = "perceptron"
const type_linearregression = "linear regression"
const type_logisticregression = "logistic regression"

In [ ]:
mutable struct SingleNeuron{ActivatorF, GradientF, LossF}
    const activationfunction::ActivatorF
    const gradient::GradientF
    const loss::LossF
    weights::Vector{Float64}
    bias::Float64
    previousweights::Vector{Float64}
    previousbias::Float64
    losshistory::Array{Float64,1}
    prevlosshistory::Array{Float64,1}
end 

In [ ]:
SingleNeuron(activationfunction::Function, 
             gradientfunction::Function, 
             lossfunction::Function, 
             weights::Vector{Float64}, 
             bias::Float64) = begin
        SingleNeuron{typeof(activationfunction), 
                    typeof(gradientfunction), 
                    typeof(lossfunction)}(activationfunction, gradientfunction, lossfunction, 
                                        weights, bias, weights, bias, [], [])
    end


In [ ]:

SingleNeuron(datadimension::Int; 
             modeltype::String, 
             weights::Vector{Float64}=zeros(Float64, datadimension), 
             bias::Float64=0.0) = begin
        if length(weights) != datadimension
            error("Provided weight vector has length $(length(weights)) \
                    not of the supplied dimension $datadimension")
        end
        if modeltype == type_perceptron
            return SingleNeuron(sign_zeropositive, perceptronstochasticgradient, 
                                perceptronloss, weights, bias)
        elseif modeltype == type_linearregression
            return SingleNeuron(linear, regressionstochasticgradient, 
                                linearregressionloss, weights, bias)
        elseif modeltype == type_logisticregression
            return SingleNeuron(sigmoid, regressionstochasticgradient, 
                                binarycrossentropyloss, weights, bias)
        else
            error("\"$modeltype\" not a recognized model.")
        end
    end

In [ ]:
perceptron = SingleNeuron(2; modeltype="perceptron")

In [ ]:
struct Test{dimension}
    a::Vector{Float64}
end

In [ ]:
a = Array{Float64,3}(undef)

In [ ]:
isconcretetype(Vector{Float64})

In [ ]:
x::Vector{Float64} = [1,2,3]

In [ ]:
typeof(x)

In [ ]:
x[3] = pi

In [ ]:
typeof(x)

In [ ]:
@show test = Test{3}([1,2,3])
@show test.a[2] = pi
@show test.af
@show isconcretetype(test.a)
b = Array{Float64, 3}(nothing, 3)
@show isconcretetype(b)
b[1] = 1

In [ ]:
typeof(regressionstochasticgradient)

In [ ]:
length([1,2,3])

In [ ]:
zeros(Float64, 5)

In [ ]:
preactivation(pred, [targ, 0], 0)

In [ ]:
func = regressionstochasticgradient

In [ ]:
@show pred = [-5, 1.2]
@show targ = [-4.9, pi]

In [ ]:
func(pred, targ)

In [ ]:
testvals = (-5, 1.2, pi)

In [ ]:
size([1...])[end]

In [ ]:
1 .- (1,2,3)

In [ ]:
using Random

In [ ]:
func = sigmoid

In [ ]:
[(func(val), val) for val in (-5, -0.1, pi)]

In [ ]:
map(val -> [sign_zeropositive(val), val], randn(5))
[(sign_zeropositive(val), val) for val in randn(5)]

In [ ]:
sign_zeropositive(0)

In [ ]:
x = 3

In [ ]:
@show x